https://kadx.co.kr/opmk/frn/pmumkproductDetail/PMU_cc3f23ff-3ac5-4f4a-90ef-a2062238856a



지역내 농식품 관련 소비를 발생시킨 고객의 성/연령별 월단위 소비금액, 소비건수 데이터

레이아웃: 기준월, 지역(광역시/도, 시군구), 성별 소비금액/건수, 성연령별 소비금액/건수
기간: 2018년 ~
성별: 남/녀
연령: 20대 미만, 20대, 30대, 40대, 50대, 60대 이상(10세 단위)

In [ ]:
# !pip install -Uq koreanize_matplotlib

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib

In [ ]:
df_col = pd.read_csv("data/kadx_data_info_detail.csv")
df_col.shape

In [ ]:
df_col

In [ ]:
raw = pd.read_csv("data/kadx_CSP_SPL-2023.csv", encoding="cp949")
raw.shape

In [ ]:
raw.head()

In [ ]:
col_dict = df_col[["nameEn","nameKo"]].set_index("nameEn")['nameKo'].to_dict()
col_dict

In [ ]:
raw = raw.rename(columns=col_dict)

In [ ]:
raw.head()

In [ ]:
raw.hist(figsize=(12, 8));

In [ ]:
raw.filter(regex="소비건수", axis=1).plot(kind="hist", bins=100, alpha=0.5, figsize=(6, 3))

In [ ]:
raw.filter(regex="(\d+대미만|\d+대|60대이상)소비금액", axis=1
          ).plot(kind="hist", bins=100, alpha=0.5, figsize=(6, 3))

In [ ]:
raw.filter(regex="소비건수", axis=1).plot(kind="kde", alpha=0.5, figsize=(6, 3));

In [ ]:
raw.filter(regex="(남성|여성)소비금액", axis=1
          ).plot(kind="density", alpha=0.5, figsize=(6, 3));

In [ ]:
raw.filter(regex="(\d+대미만|\d+대|60대이상)소비금액", axis=1
          ).plot(kind="hist", bins=100, alpha=0.5, subplots=True, figsize=(12, 8));

In [ ]:
corr = raw.corr(numeric_only=True)
plt.figure(figsize=(10, 10))
sns.heatmap(corr, annot=True, cmap="coolwarm")

In [ ]:
# Tidy data로 변환
tidy_df = pd.melt(
    raw,
    id_vars=['기준연월', '소비지역광역시도명', '소비지역시군구명'],
    var_name='성별_연령대_항목',
    value_name='값'
)

# '성별', '연령대', '항목' 컬럼으로 분리
tidy_df[['성별', '연령대', '항목']] = tidy_df['성별_연령대_항목'].str.extract(r'(남성|여성)(\d+대미만|\d+대|60대이상)?(소비금액|소비건수)')
tidy_df

In [ ]:
# 불필요한 컬럼 제거
tidy_df = tidy_df.drop(columns=['성별_연령대_항목'])

# tidy data 출력
tidy_df.head()

In [ ]:
tidy_df["항목"].value_counts()

In [ ]:
tidy_df.groupby(["항목"])["값"].describe()

In [ ]:
# 소비금액과 소비건수를 컬럼으로 변환
df = tidy_df.pivot(
    index=['기준연월', '소비지역광역시도명', '소비지역시군구명', '성별', '연령대'],
    columns='항목'
).reset_index()
col_name = []
for col in df.columns:
    col_name.append("".join(col).replace("값", ""))
col_name

In [ ]:
df.columns = col_name

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe().style.format("{:,.0f}")

In [ ]:
df.describe(exclude="number")

In [ ]:
df.hist(bins=50, figsize=(12, 3));

In [ ]:
df.groupby(["성별"])["소비금액"].describe().style.format("{:,.0f}")

In [ ]:
gender_age = df.groupby(["연령대", "성별"])["소비금액"].describe()
gender_age.style.format("{:,.0f}")

In [ ]:
df["연령대"].unique()

In [ ]:
age_order = ['20대미만', '20대', '30대', '40대', '50대', '60대이상']

In [ ]:
gender_age.loc[age_order].style.background_gradient(cmap="Blues").format("{:,.0f}")